# A Star 8 Puzzles

## Homework
ให้เลือกสถานะตั้งต้นของ 8-Puzzle มาหนึ่งอัน เปรียบเทียบจำนวน node ที่ถูก generate ขึ้นเมื่อใช้ h1 และ h2

ให้นิสิตเขียน code แยกเป็นสอง cell โดย cell หนึ่งเป็น h1 และอีก cell เป็น h2 โดยทั้งสอง cell แสดงจำนวน node ที่ถูก generate ขึ้นจากการ search โดยใช้ h1 และ h2 เป็น heuristic

## Original Code

### **Informed Search**

**h1 (state, cost)**

  คำนวณค่า heuristic แบบ misplaced tile โดยนับจำนวนตัวเลขที่วางผิดตำแหน่งเมื่อเทียบกับ goal state

**manhattan(state, goal, ind)**

   คำนวณระยะห่าง Manhattan ระหว่างตำแหน่งปัจจุบันของตัวเลขที่ตำแหน่ง ind ใน state กับตำแหน่งที่ควรจะเป็นใน goal_state

**h2 (state, cost)**


คำนวณค่า heuristic แบบ total Manhattan distance โดยรวมระยะห่าง Manhattan ของทุกตัวเลขที่วางผิดตำแหน่ง

**gen_successors(node)**

สร้าง list ของ node ลูกที่เป็นไปได้ทั้งหมดจาก node ปัจจุบัน โดยพิจารณาการเคลื่อนที่ของช่องว่างไปใน 4 ทิศทาง (ขึ้น ลง ซ้าย ขวา)

**is_goal(node)**

ตรวจสอบว่า node ปัจจุบันตรงกับ goal state หรือไม่

**insert_all(node, fringe)**

สร้าง node ลูกจาก node ปัจจุบัน แล้วใส่ node ลูกเหล่านี้เข้าไปใน fringe โดยเรียงลำดับตามค่า f = g + h

**show_result(g, visited_node)**

แสดงลำดับขั้นตอนการแก้ปริศนาจาก node เริ่มต้นไปจนถึง goal state โดยไล่ย้อนกลับไปตาม parent node

**a_star(start_node)**

อัลกอริทึม A* search สำหรับแก้ปริศนา 8-Puzzle โดยใช้ heuristic ที่กำหนด


In [ ]:
last_index = 0
visited_node = {}
goal_state = '8123456780'
total_nodes = 0
total_expanded_nodes = 0

## h1
def h1(state,cost): # misplaced tile
  count = 0
  for i in range(1,10):
    if state[i] == '0':
      continue
    if state[i] != goal_state[i]:
      count += 1
  return count+cost

def manhattan(state,goal,ind):
  j = goal[1:].find(state[ind+1])
  rj = j//3
  cj = j%3
  ri = ind//3
  ci = ind%3
    # print(j,rj,cj,(i-1),ri,ci)
  return abs(rj-ri) + abs(cj-ci)

## h2
def h2(state,cost): # total Manhattan distance
  count = 0
  for i in range(1,10):
    if state[i] == '0':
      continue
    count += manhattan(state,goal_state,i-1)
  return count + cost

def gen_successors(node):
    global last_index
    node_id,level,cost,f = node[1],node[3],node[4],node[5]
    ret = []
    state = node[0]
    loc = int(state[0])
    puzzle = state[1:]
    if loc % 3 != 0:
        new_state = str(loc-1) + puzzle[:loc-1] + puzzle[loc] + puzzle[loc-1] + puzzle[loc+1:]
        ret += [(new_state,last_index+1,node[1],level+1,cost+1,h1(new_state,cost+1))]
        last_index += 1
        # print('L'+new_state)
    if (loc+1) % 3 != 0:
        new_state = str(loc+1) + puzzle[:loc] + puzzle[loc+1] + puzzle[loc] + puzzle[loc+2:]
        ret += [(new_state,last_index+1,node[1],level+1,cost+1,h1(new_state,cost+1))]
        last_index += 1
        # print('R'+new_state)
    if loc >= 3:
        new_state = str(loc-3) + puzzle[:loc-3] + puzzle[loc] + puzzle[loc-2:loc] + puzzle[loc-3] + puzzle[loc+1:]
        ret += [(new_state,last_index+1,node[1],level+1,cost+1,h1(new_state,cost+1))]
        last_index += 1
        # print('U'+new_state)
    if loc <= 5:
        new_state = str(loc+3) + puzzle[:loc] + puzzle[loc+3] + puzzle[loc+1:loc+3] + puzzle[loc] + puzzle[loc+4:]
        ret += [(new_state,last_index+1,node[1],level+1,cost+1,h1(new_state,cost+1))]
        last_index += 1
        # print('D'+new_state)
    return ret

def is_goal(node):
    return node[0] == '8123456780'

def insert_all(node,fringe):
    global total_nodes
    children = gen_successors(node)
    total_nodes += len(children)
    for child in children:
      inserted = False
      for j in range(len(fringe)):
        if child[4]+child[5] < fringe[j][4]+fringe[j][5]:
          fringe[j:j] = [child]
          inserted = True
          break
      if not inserted:
        fringe.append(child)
    # print(fringe)

def show_result(g,visited_node):
    current_node = g
    parent_index = current_node[1]
    while True:
        #print(current_node[0][1:4])
        #print(current_node[0][4:7])
        #print(current_node[0][7:])
        #print()
        parent_index = current_node[2]
        if parent_index == -1:
            break
        current_node = visited_node[parent_index]

def a_star(start_node):
    global last_index
    global visited_node
    global total_expanded_nodes

    last_index = 0
    fringe = [start_node]
    # print(fringe)
    visited_node = {}
    while True:
        if len(fringe) == 0:
            # print('Not Found')
            break
        front = fringe[0]
        visited_node[front[1]] = front
        fringe = fringe[1:]

        total_expanded_nodes += 1
        if is_goal(front):
            show_result(front,visited_node)
            return
        insert_all(front,fringe)

In [ ]:
a_star(('2170532486', 0, -1, 0, 0, 0)) # (state, node_id, parent_id, depth, cost, f) # 3135072486 1401852736
print(total_expanded_nodes)

11738


In [ ]:
('2170532486',0,-1,0,0,0)[0]

'2170532486'

### **gen_start function**

สร้างสถานะเริ่มต้น (start state) ของปริศนา 8-Puzzle แบบสุ่ม โดยเริ่มจากสถานะเป้าหมาย (goal state) แล้วทำการสลับตำแหน่งตัวเลขแบบสุ่มเป็นจำนวนครั้งตามที่กำหนด (step)

In [ ]:
import random

def gen_start(state, step):
  for i in range(step):
    loc0 = int(state[0])
    dir = random.randint(0, 3)
    if dir == 0 and loc0 >= 3:
      new_loc0 = loc0 - 3
    elif dir == 1 and loc0 <= 5:
      new_loc0 = loc0 + 3
    elif dir == 2 and loc0%3 != 0:
      new_loc0 = loc0 - 1
    elif dir == 3 and (loc0+1)%3 != 0:
      new_loc0 = loc0 + 1
    else:
      new_loc0 = loc0

    newstate = list(state[1:])
    newstate[loc0],newstate[new_loc0] = newstate[new_loc0],newstate[loc0]
    newstate = str(new_loc0)+''.join(newstate)
    state = newstate

  return newstate

goal_state = '8123456780'
gen_start(goal_state, 60)

'1102563478'

# FROM TA

In [ ]:
start_state = '2170532486'

# Reset variables for h1
last_index = 0
visited_node = {}
total_nodes = 0
total_expanded_nodes = 0

# A* Search with h1 (Misplaced Tiles) Heuristic
a_star( ( start_state, 0, -1, 0, 0, h1(start_state, 0) ) )

print("Results using h1 heuristic:")
print("Total expanded nodes:", total_expanded_nodes) # 11738
print("Total nodes generated:", total_nodes) # 32803

Results using h1 heuristic:
Total expanded nodes: 11738
Total nodes generated: 32803


In [ ]:
# Reset variables for h2
last_index = 0
visited_node = {}
total_nodes = 0
total_expanded_nodes = 0

# A* Search with h2 (Manhattan Distance) Heuristic
a_star( ( start_state, 0, -1, 0, 0, h2(start_state, 0) ) )  # Use the same start_state

print("Results using h2 heuristic:")
print("Total expanded nodes:", total_expanded_nodes) #11738
print("Total nodes generated:", total_nodes) # 32803

Results using h2 heuristic:
Total expanded nodes: 11738
Total nodes generated: 32803


## Test Case set

In [ ]:
start_state = gen_start(goal_state, 60)
start_state

'6451872063'

In [ ]:
def h1_test(start_state):
  # Reset variables for h1
  last_index = 0
  visited_node = {}
  total_nodes = 0
  total_expanded_nodes = 0

  # A* Search with h1 (Misplaced Tiles) Heuristic
  a_star((start_state, 0, -1, 0, 0, h1(start_state, 0)))

  print("Results using h1 heuristic:")
  print("Total expanded nodes:", total_expanded_nodes)
  print("Total nodes generated:", total_nodes)

In [ ]:
def h2_test(start_state):
  # Reset variables for h2
  last_index = 0
  visited_node = {}
  total_nodes = 0
  total_expanded_nodes = 0

  # A* Search with h2 (Manhattan Distance) Heuristic
  a_star((start_state, 0, -1, 0, 0, h2(start_state, 0)))  # Use the same start_state

  print("\nResults using h2 heuristic:")
  print("Total expanded nodes:", total_expanded_nodes)
  print("Total nodes generated:", total_nodes)

In [ ]:
print(start_state)
h1_test(start_state)
h2_test(start_state)

6451872063
Results using h1 heuristic:
Total expanded nodes: 0
Total nodes generated: 0

Results using h2 heuristic:
Total expanded nodes: 0
Total nodes generated: 0


In [ ]:
## start_state = '5465310782'
h1_test('5465310782')
h2_test('5465310782')

In [ ]:
# 1401852736
h1_test('1401852736')
h2_test('1401852736')

In [ ]:
# 3135072486
h1_test('3135072486')
h2_test('3135072486')